In [242]:
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import ast
load_dotenv()

data_folder = os.getenv("DATA_FOLDER")

iteration = 0
test_df = pd.read_csv(f'{data_folder}/test_label_data.csv', sep=";",index_col=0)
df = pd.read_csv(f'{data_folder}/label_data_{iteration*100}_{iteration*100+100}.csv', sep=";",index_col=0)
json_path = f'{data_folder}/label_data_{iteration*100}_{iteration*100+100}.json'


In [243]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, recall_score, precision_score, ndcg_score
from scipy.optimize import minimize
from scipy.special import expit


In [244]:
test_df['Labels']

0    [{'comparisons': [{'claim_article_a': 'Arizona...
Name: Labels, dtype: object

In [245]:
faulty_labels = 0
def safe_literal_eval(x):
    global faulty_labels
    try:
        return ast.literal_eval(x)
    except:
        faulty_labels += 1


In [246]:
df['Labels'] = df['Labels'].apply(safe_literal_eval)
print(faulty_labels)

0


In [247]:
df['Labels']

0     [{'comparisons': [{'claim_article_a': "Arizona...
1     [{}, {}, {}, {}, {}, {}, {}, {}, {}, {'compari...
2     [{'comparisons': [{'claim_article_a': "Arizona...
3     [{}, {}, {'comparisons': [{'claim_article_a': ...
4     [{}, {}, {'comparisons': [{'claim_article_a': ...
                            ...                        
95    [{'comparisons': [{'claim_article_a': 'Dr. Rob...
96                 [{}, {}, {}, {}, {}, {}, {}, {}, {}]
97             [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
98                 [{}, {}, {}, {}, {}, {}, {}, {}, {}]
99             [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
Name: Labels, Length: 100, dtype: object

In [248]:
def get_num_articles(label_lists):
    return sum(bool(d) for d in label_lists)

df['num_valid_articles'] = df['Labels'].apply(get_num_articles)

In [249]:
df['num_valid_articles'].describe()

count    100.000000
mean       2.470000
std        1.982015
min        0.000000
25%        1.000000
50%        2.000000
75%        4.000000
max        9.000000
Name: num_valid_articles, dtype: float64

If a Newsguard Score could not be retrieved for an article, it will be filled by the placeholder score, in this case the median of the average newsguard scores as given by Aslett et al.

In [250]:
placeholder_ng_score = df['avg_score'].median()
placeholder_ng_score

np.float64(89.6875)

In [251]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
def weighted_metric(weights, label_dict_list, positions=False):
    #create three distinct lists to control each parameter
    if not label_dict_list:
        return None
    article_stances = []
    article_ng_scores = []
    article_serp_positions = []
    
    for article in label_dict_list: #this is the search level; here we can retrieve SERP position and Newsguard score
        if not article: #catch empty entries
            continue
        if len(article['comparisons']) == 0:
               continue
        #retrieve variables
        serp_newsguard = article['serp_newsguard']
        serp_position = article['serp_position']

        #add to lists for decomposed calculation
        article_ng_scores.append(serp_newsguard)
        article_serp_positions.append(serp_position)

        article_stance = 0
        for claim_pair in article['comparisons']: #this is the "article level"; here we can retrieve relative claim position and the bool headline info
            relation = claim_pair['relation']
            if relation == 0:
                continue

            try:
                #retrieve relative claim positions
                fm_claim_position = claim_pair['fm_claim_position']
                serp_claim_position = claim_pair['serp_claim_position']

                #retrieve headline bool
                fm_claim_headline = claim_pair['fm_claim_headline']
                serp_claim_headline = claim_pair['serp_claim_headline']
            except:
                if positions:
                    return None

            article_stance += relation

        article_stances.append(article_stance/len(article['comparisons']))
        article_stance = 0

    if not article_stances:
        return None

    article_stances = np.array(article_stances)
    article_ng_scores = np.array(article_ng_scores, dtype=float)
    article_ng_scores = np.nan_to_num(article_ng_scores, nan=placeholder_ng_score) #replace NaN with the placeholder
    article_ng_scores = article_ng_scores / 100 #normalize newsguard scores between 0 and 1
    article_serp_positions = np.array(article_serp_positions)
    article_serp_positions = 1/article_serp_positions #1/x


    # print(article_stances)
    # print(article_ng_scores)
    # print(article_serp_positions)
    metrics =  np.vstack((article_stances, article_ng_scores, article_serp_positions))
    weighted_preds = np.dot(weights, metrics)
    return sigmoid(np.median(weighted_preds))


In [267]:
label_mapping = {
    'Coul' : np.nan,
    'Misl' : 0,
    'True' : 1
}

df['User_Labels'] = df['Category'].map(label_mapping)

In [268]:
df = df[df['User_Labels'].notna()]
df

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls,newsguard_dict,Labels,num_valid_articles,User_Labels
1,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,0.0,0.0,95.277778,"abc15.com', 'salon.com', 'cnn.com', 'politico....","100,87.5,87.5,100,95,95,100,100,92.5",('https://www.abc15.com/news/state/poll-many-r...,"{'abc15.com': 100, 'salon.com': 87.5, 'cnn.com...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {'compari...",1,0.0
2,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,1.0,1.0,83.450000,"cnn.com', 'thehill.com', 'salon.com', 'washing...","87.5,80,87.5,42,87.5,100,92.5,87.5,85,85",('https://www.cnn.com/2021/07/18/politics/fact...,"{'cnn.com': 87.5, 'thehill.com': 80, 'salon.co...","[{'comparisons': [{'claim_article_a': ""Arizona...",4,0.0
3,Day_1_1,R_3KT6q7Vntwvmg8Z,Day_1,True,1,7,3,33,1,FM,...,0.0,0.0,92.812500,"thehill.com', 'recorder.maricopa.gov', 'washin...","80,100,100,100,100,92.5,75,95",('https://thehill.com/homenews/campaign/563100...,"{'thehill.com': 80, 'recorder.maricopa.gov': 1...","[{}, {}, {'comparisons': [{'claim_article_a': ...",6,1.0
4,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,0.0,0.0,96.000000,"apnews.com', 'detroitnews.com', 'cnn.com', 'ap...","95,92.5,87.5,95,95,100,100,100,100,95",('https://apnews.com/article/technology-joe-bi...,"{'apnews.com': 95, 'detroitnews.com': 92.5, 'c...","[{}, {}, {'comparisons': [{'claim_article_a': ...",6,0.0
5,Day_1_1,R_yx69bWSwS7KJTDb,Day_1,Misl,0,3,2,25,1,FM,...,0.0,0.0,NaN,"twitter.com', 'twitter.com', 'beckernews.com',...",NaN,('https://twitter.com/kylenabecker?ref_src=tws...,{},"[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]",0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Day_1_3,R_24d72uJHVnyQYtL,Day_1,True,1,6,4,67,0,FM,...,1.0,1.0,56.000000,"techstartups.com', 'twitter.com', 'zerohedge.c...","5,7.5,87.5,87.5,92.5",('https://techstartups.com/2021/07/17/mrna-vac...,"{'techstartups.com': 5, 'twitter.com': 7.5, 'z...","[{}, {}, {}, {}, {}, {}, {}, {}, {'comparisons...",1,1.0
95,Day_1_3,R_6gSAhAKCKeijlQZ,Day_1,True,1,6,4,39,0,FM,...,0.0,0.0,91.500000,"newportri.com', 'galvnews.com', 'fortune.com',...","95,92.5,100,87.5,82.5",('https://www.newportri.com/story/opinion/lett...,{},[{'comparisons': [{'claim_article_a': 'Dr. Rob...,2,1.0
96,Day_1_3,R_1Cr6QLqQcjGmEWV,Day_1,Misl,0,2,1,53,0,FM,...,1.0,1.0,53.750000,"tigerdroppings.com', 'teamblind.com', 'glockta...","100,7.5",('https://www.tigerdroppings.com/rant/politics...,"{'tigerdroppings.com': 100, 'teamblind.com': 7.5}","[{}, {}, {}, {}, {}, {}, {}, {}, {}]",0,0.0
97,Day_1_3,R_XilsxctsD6UhvsR,Day_1,Misl,0,3,2,37,0,FM,...,0.0,0.0,98.571429,"googleadservices.com', 'youtube.com', 'nytimes...","100,100,100,100,95,95,100","('https://www.googleadservices.com/', 'https:/...","{'googleadservices.com': 100, 'youtube.com': 1...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]",0,0.0


In [ ]:
def cost_function(weights, label_list, y):
    pred_list = np.array([weighted_metric(weights, l) for l in label_list]) 
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    #normalized_preds = scaler.fit_transform(pred_list.reshape(-1, 1)).flatten()
    print(pred_list)
    y = np.array(y)
    y = y[~np.isnan(pred_list)]
    print(y)
    pred_list = pred_list[~np.isnan(pred_list)]
    return mean_squared_error(y, pred_list)

In [274]:
initial_weights = np.array([0.33, 0.33, 0.33])
cost_function(initial_weights, df['Labels'].tolist(), df['User_Labels'].tolist())

[0.5815084366955354 0.5863499401646062 0.6139448812010134
 0.5547113092946534 None 0.6336250237341008 0.5815084366955354
 0.585401932587979 0.585401932587979 0.5919990232731771 0.5919990232731771
 0.6322194183501241 0.585368162754207 0.5723633180440735
 0.6149302535446237 0.6336250237341008 0.5931838104128396
 0.5962872129694232 0.5653659357636591 None 0.5785441902836977
 0.6262046431017451 0.6577762353687765 0.6015278955704818
 0.5868524785397771 0.5902182561614635 0.5931055955600439
 0.5815084366955354 0.619729440865097 0.6052406956812597
 0.5271152984312819 0.5928898877323376 0.6205796753721665
 0.6006486982741097 0.5857685815180886 0.6104846795158617
 0.5683733935871558 0.5905646263614605 0.5873857810469977
 0.581988770808365 0.578619623606993 0.5957409404667504 0.5822945742491356
 0.5935037310307851 0.5893616455085824 0.5289775359476135
 0.5734567497524867 0.5877689656597637 0.578619623606993
 0.5171521213149903 0.5966236863881923 0.5825119429172573 None
 0.6073276085512612 0.5950

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [240]:
initial_weights = np.array([0.33, 0.33, 0.33]) #Weights for article stance, ng score, serp position

# def weight_constraint(weights):
#     return np.sum(weights) - 1
# constraints = ({'type': 'eq', 'fun': weight_constraint})

result = minimize(cost_function, initial_weights, args=(df['Labels'].tolist(), df['User_Labels'].tolist()), method='SLSQP', bounds=[(0, 1), (0, 1), (0, 1)], options={'maxiter': 1000, 'ftol': 1e-6})

optimal_weights = result.x
print(f"Optimal Weights: {optimal_weights}")

cost_function(optimal_weights, df['Labels'].tolist(), df['User_Labels'])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
def get_metrics(weights, label_list):
    pred_list = np.array([weighted_metric(weights, l) for l in label_list])
    return pred_list
    # scaler = MinMaxScaler(feature_range=(-1, 1))
    # normalized_preds = scaler.fit_transform(pred_list.reshape(-1, 1)).flatten()
    # return normalized_preds

In [ ]:
df ['Metric'] = get_metrics(optimal_weights, df['Labels'].tolist())

In [ ]:
df['Metric']

1     0.513497
2     0.545626
3     0.552892
4     0.471542
6     0.599891
        ...   
90    0.526975
91    0.421664
93    0.535021
94    0.514996
95    0.543785
Name: Metric, Length: 74, dtype: float64

In [ ]:
df = df[df['Metric'].notna()]

KeyError: 'Metric'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, recall_score, precision_score

In [ ]:
# Calculate Mean Squared Error
mse = mean_squared_error(df['User_Labels'], df['Normalized_Metric'])
print(f"Mean Squared Error: {mse}")

# Calculate Mean Absolute Error
mae = mean_absolute_error(df['User_Labels'], df['Normalized_Metric'])
print(f"Mean Absolute Error: {mae}")

# Calculate R-squared
r2 = r2_score(df['User_Labels'], df['Normalized_Metric'])
print(f"R-squared: {r2}")


Mean Squared Error: 0.42629269246023915
Mean Absolute Error: 0.5613075171310218
R-squared: -0.7101676072617364


In [ ]:
df['Binary_Metric'] = [1 if (x) > 0.5 else 0 for x in df['Metric']]

In [ ]:
df['Binary_Metric']

1     1
2     1
3     1
4     0
6     1
     ..
90    1
91    0
93    1
94    1
95    1
Name: Binary_Metric, Length: 74, dtype: int64

In [ ]:
# Calculate Accuracy
acc = accuracy_score(df['User_Labels'], df['Binary_Metric'])
print(f"Accuracy: {acc}")

# Calculate F1
f1 = f1_score(df['User_Labels'], df['Binary_Metric'], average="micro")
print(f"F1: {f1}")

# Calculate Precision
prec = precision_score(df['User_Labels'], df['Binary_Metric'], average="micro")
print(f"Precision: {prec}")

# Calculate Recall
recall = recall_score(df['User_Labels'], df['Binary_Metric'], average="micro")
print(f"Recall: {recall}")

Accuracy: 0.581081081081081
F1: 0.581081081081081
Precision: 0.581081081081081
Recall: 0.581081081081081
